In [1]:
%env GEOMSTATS_BACKEND=numpy

%env NUMEXPR_MAX_THREADS=12 

import pickle
import nu_smrutils as u
import pandas as pd
import numpy as np

import geomstats.backend as gs
import geomstats.geometry.spd_matrices as spd
from geomstats.learning.frechet_mean import FrechetMean
from geomstats.learning.preprocessing import ToTangentSpace

from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

env: GEOMSTATS_BACKEND=numpy
env: NUMEXPR_MAX_THREADS=12


INFO: Using numpy backend


In [18]:
class RMLKNN:
    def __init__(self, subjects, epochs, klist=[3,5,7,9], metrics=['AIRM','LEM']):
        self.subjects = subjects
        self.klist=klist
        self.metrics=metrics
        self.epochs = epochs
        
    def outer_LOSO(self, pairwise_d, l):
        results =[]
        for i in range(self.subjects):
            print("Subject: "+str(i+1))
            remain_d = []
            for j in range(len(pairwise_d)):
                remain_d.append(np.delete(np.delete(pairwise_d[j], slice(i*self.epochs, (i+1)*self.epochs), 0), slice(i*self.epochs, (i+1)*self.epochs), 1)) 
            remain_l = l[:i*self.epochs] + l[(i+1)*self.epochs:]
            [k, m] = self.inner_LOSO(remain_d, remain_l)
            
            print("Choice for the subject is: K = " + str(self.klist[k]) + " Metric = " + str(self.metrics[m]))
            
            clf = KNN(n_neighbors = self.klist[k], metric = 'precomputed')
            res = cross_validate(clf, pairwise_d[m], l, cv=self.subjects)
            acc = res['test_score'][i]
            results.append(acc)
            print("Accuracy for this subject is: " + str(acc))
            
        return results
            
    def inner_LOSO(self, pairwise_d, l): 
        max_acc = 0
        max_indecies = [0, 0]
        for k in range(len(self.klist)):
            clf = KNN(n_neighbors = self.klist[k], metric = 'precomputed')
            for m in range(len(pairwise_d)):
                acc = cross_val_score(clf, pairwise_d[m], l, cv=self.subjects-1)
                if acc.mean()>max_acc:
                    max_acc = acc.mean()
                    max_indecies = [k, m]
        return max_indecies

In [19]:
d_filename='datasets/SPDDataset.pickle'
infile=open(d_filename,'rb')
[d, l] = pickle.load(infile)

k_filename = 'datasets/pairwise10.pickle'
infile=open(k_filename,'rb')
pairwise_d = pickle.load(infile)

In [20]:
subjects = 10
channels = 62 
epochs = 400 #total epochs in one subject

d_sub = d[:10*epochs]
l_sub = l[:10*epochs]

In [21]:
a = RMLKNN(subjects, epochs)

In [22]:
results = a.outer_LOSO(pairwise_d, l_sub)

Subject: 1
Choice for the subject is: K = 5 Metric = AIRM
Accuracy for this subject is: 0.5075
Subject: 2
Choice for the subject is: K = 7 Metric = AIRM
Accuracy for this subject is: 0.53
Subject: 3
Choice for the subject is: K = 3 Metric = AIRM
Accuracy for this subject is: 0.54
Subject: 4
Choice for the subject is: K = 3 Metric = AIRM
Accuracy for this subject is: 0.5375
Subject: 5
Choice for the subject is: K = 3 Metric = AIRM
Accuracy for this subject is: 0.46
Subject: 6
Choice for the subject is: K = 3 Metric = AIRM
Accuracy for this subject is: 0.5125
Subject: 7
Choice for the subject is: K = 7 Metric = AIRM
Accuracy for this subject is: 0.51
Subject: 8
Choice for the subject is: K = 3 Metric = AIRM
Accuracy for this subject is: 0.4925
Subject: 9
Choice for the subject is: K = 3 Metric = AIRM
Accuracy for this subject is: 0.5175
Subject: 10
Choice for the subject is: K = 9 Metric = AIRM
Accuracy for this subject is: 0.495


In [ ]:
"""
ai_metric=spd.SPDMetricAffine(n=channels)
le_metric=spd.SPDMetricLogEuclidean(n=channels)
AI_pairwise=ai_metric.dist_pairwise(d_sub)
LE_pairwise=le_metric.dist_pairwise(d_sub)
pairwise_d =[AI_pairwise, LE_pairwise]
filename = 'datasets/pairwise10.pickle'
outfile = open(filename,'wb')
pickle.dump(pairwise, outfile)
outfile.close()
"""